In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# from skfeature.function.similarity_based import fisher_score
import warnings
# warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

# EDA

In [2]:
df = pd.read_csv('OnlineNewsPopularity.csv')
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.148948,0.043137,0.015686,0.733333,0.266667,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,918.0,918.0,918.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.323333,0.056872,0.009479,0.857143,0.142857,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,4.404896,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028573,0.419300,0.494651,0.028905,0.028572,0.429850,0.100705,0.041431,0.020716,0.666667,0.333333,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,4.682836,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,545.0,16000.0,3151.157895,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028633,0.028794,0.028575,0.028572,0.885427,0.513502,0.281003,0.074627,0.012127,0.860215,0.139785,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [3]:
df.describe()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
count,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000
mean,354.530471,10.398749,546.514731,0.548216,0.996469,0.689175,10.883690,3.293638,4.544143,1.249874,4.548239,7.223767,0.052946,0.178009,0.157855,0.058597,0.185299,0.212567,26.106801,1153.951682,312.366967,13612.354102,752324.066694,259281.938083,1117.146610,5657.211151,3135.858639,3998.755396,10329.212662,6401.697580,0.168020,0.186409,0.187544,0.183306,0.143805,0.061876,0.069039,0.130915,0.184599,0.141256,0.216321,0.223770,0.234029,0.443370,0.119309,0.039625,0.016612,0.682150,0.287934,0.353825,0.095446,0.756728,-0.259524,-0.521944,-0.107500,0.282353,0.071425,0.341843,0.156064,3395.380184
std,214.163767,2.114037,471.107508,3.520708,5.231231,3.264816,11.332017,3.855141,8.309434,4.107855,0.844406,1.909130,0.223929,0.382525,0.364610,0.234871,0.388545,0.409129,69.633215,3857.990877,620.783887,57986.029357,214502.129573,135102.247285,1137.456951,6098.871957,1318.150397,19738.670516,41027.576613,24211.332231,0.373889,0.389441,0.390353,0.386922,0.350896,0.240933,0.253524,0.337312,0.262975,0.219707,0.282145,0.295191,0.289183,0.116685,0.096931,0.017429,0.010828,0.190206,0.156156,0.104542,0.071315,0.247786,0.127726,0.290290,0.095373,0.324247,0.265450,0.188791,0.226294,11626.950749
min,8.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.393750,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,1.000000
25%,164.000000,9.000000,246.000000,0.470870,1.000000,0.625739,4.000000,1.000000,1.000000,0.000000,4.478404,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,445.000000,141.750000,0.000000,843300.000000,172846.875000,0.000000,3562.101631,2382.448566,639.000000,1100.000000,981.187500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025051,0.025012,0.028571,0.028571,0.0

In [4]:
#Check Nullity
df.isnull().sum() #It appears that the data has no null values

url                               0
 timedelta                        0
 n_tokens_title                   0
 n_tokens_content                 0
 n_unique_tokens                  0
 n_non_stop_words                 0
 n_non_stop_unique_tokens         0
 num_hrefs                        0
 num_self_hrefs                   0
 num_imgs                         0
 num_videos                       0
 average_token_length             0
 num_keywords                     0
 data_channel_is_lifestyle        0
 data_channel_is_entertainment    0
 data_channel_is_bus              0
 data_channel_is_socmed           0
 data_channel_is_tech             0
 data_channel_is_world            0
 kw_min_min                       0
 kw_max_min                       0
 kw_avg_min                       0
 kw_min_max                       0
 kw_max_max                       0
 kw_avg_max                       0
 kw_min_avg                       0
 kw_max_avg                       0
 kw_avg_avg                 

In [5]:
#Drop columns that are obbiously not going to be useful
df.drop(['url', ' timedelta', ' n_non_stop_words', ' n_non_stop_unique_tokens'], axis=1, inplace=True)

In [6]:
#25 Percentile
print(df[' shares'].quantile(q=0.25))

#50 Percentile
print(df[' shares'].quantile(q=0.50))

#75 Percentile
print(df[' shares'].quantile(q=0.75))


946.0
1400.0
2800.0


In [7]:
#Convert ' share' to categorical
# <25 percentile: poor, 25-75 percentile: average, 75-100 percentile: good
share_label = list()
for share in df[' shares']:
#     if share <= 946:
#         share_label.append('Poor')
#     elif share > 946 and share <= 2800:
#         share_label.append('Average')
#     else:
#         share_label.append('Good')
        
    if share <= 946:
        share_label.append(0)#Very Poor
    elif share > 946 and share <= 1400:
        share_label.append(1)#Poor
    elif share >1400 and share <= 2800:
        share_label.append(2)#Above Average
    else:
        share_label.append(3)#Good
        
        
#     if share <= 946:
#         share_label.append('Poor')
#     else:
#         share_label.append('Good')

        
#     if share <= 645:
#         share_label.append('Very Poor')
#     elif share > 645 and share <= 861:
#         share_label.append('Poor')
#     elif share > 861 and share <= 1400:
#         share_label.append('Average')
#     elif share > 1400 and share <= 31300:
#         share_label.append('Good')
#     elif share > 31300 and share <= 53700:
#         share_label.append('Very Good')
#     elif share > 53700 and share <= 77200:
#         share_label.append('Excellent')
#     else:
#         share_label.append('Exceptional')

# Update this class label into the dataframe
df = pd.concat([df, pd.DataFrame(share_label, columns=['popularity'])], axis=1)


In [8]:
#Some of the features are related to each other such as ' weekday_is_monday' and ' weekday_is_tuesday' and ' data_channel_is_lifestyle' and ' data_channel_is_entertainment'. This violates non-multicollinearity of model such as linear regression
#Merging 
dayMerge = df[[' weekday_is_monday',' weekday_is_tuesday',' weekday_is_wednesday',' weekday_is_thursday',' weekday_is_friday',' weekday_is_saturday',' weekday_is_sunday', ]]
day_arr = []
for r in list(range(dayMerge.shape[0])):
    for c in list(range(dayMerge.shape[1])):
        if ((c==0) and (dayMerge.iloc[r,c])==1):
            day_arr.append(0)
        elif ((c==1) and (dayMerge.iloc[r,c])==1):
            day_arr.append(1)
        elif ((c==2) and (dayMerge.iloc[r,c])==1):
            day_arr.append(2)
        elif ((c==3) and (dayMerge.iloc[r,c])==1):
            day_arr.append(3)
        elif ((c==4) and (dayMerge.iloc[r,c])==1):
            day_arr.append(4)
        elif ((c==5) and (dayMerge.iloc[r,c])==1):
            day_arr.append(5) 
        elif ((c==6) and (dayMerge.iloc[r,c])==1):
            day_arr.append(6)
            
channelMerge=df[[' data_channel_is_lifestyle',' data_channel_is_entertainment' ,' data_channel_is_bus',
                        ' data_channel_is_socmed' ,' data_channel_is_tech',' data_channel_is_world' ]]

channel_arr = []

for r in list(range(channelMerge.shape[0])):
    if (((channelMerge.iloc[r,0])==0) and ((channelMerge.iloc[r,1])==0) and ((channelMerge.iloc[r,2])==0) and ((channelMerge.iloc[r,3])==0) and ((channelMerge.iloc[r,4])==0) and ((channelMerge.iloc[r,5])==0)):
        channel_arr.append(6)
    for c in list(range(channelMerge.shape[1])):
        if ((c==0) and (channelMerge.iloc[r,c])==1):
            channel_arr.append(0)
        elif ((c==1) and (channelMerge.iloc[r,c])==1):
            channel_arr.append(1)
        elif ((c==2) and (channelMerge.iloc[r,c])==1):
            channel_arr.append(2)
        elif ((c==3) and (channelMerge.iloc[r,c])==1):
            channel_arr.append(3)
        elif ((c==4) and (channelMerge.iloc[r,c])==1):
            channel_arr.append(4)
        elif ((c==5) and (channelMerge.iloc[r,c])==1):
            channel_arr.append(5)
            
df.insert(loc=11, column='weekdays', value=day_arr)
df.insert(loc=12, column='data_channel', value=channel_arr)

df.drop(labels=[' data_channel_is_lifestyle',' data_channel_is_entertainment' ,' data_channel_is_bus',
                  ' data_channel_is_socmed' ,' data_channel_is_tech',' data_channel_is_world', 
                  ' weekday_is_monday',' weekday_is_tuesday',' weekday_is_wednesday', 
                  ' weekday_is_thursday', ' weekday_is_friday',' weekday_is_saturday' ,' weekday_is_sunday', ' shares'], axis = 1, inplace=True)

In [9]:
df.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,weekdays,data_channel,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,popularity
0,12.0,219.0,0.663594,4.0,2.0,1.0,0.0,4.680365,5.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.000000,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,0
1,9.0,255.0,0.604743,3.0,1.0,1.0,0.0,4.913725,4.0,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.148948,0.043137,0.015686,0.733333,0.266667,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,0
2,9.0,211.0,0.575130,3.0,1.0,1.0,0.0,4.393365,6.0,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,918.0,918.0,918.000000,0.0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.323333,0.056872,0.009479,0.857143,0.142857,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,2
3,9.0,531.0,0.503788,9.0,0.0,1.0,0.0,4.404896,7.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.028573,0.419300,0.494651,0.028905,0.028572,0.429850,0.100705,0.041431,0.020716,0.666667,0.333333,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1
4,13.0,1072.0,0.415646,19.0,19.0,20.0,0.0,4.682836,7.0,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,545.0,16000.0,3151.157895,0.0,0.028633,0.028794,0.028575,0.028572,0.885427,0.513502,0.281003,0.074627,0.012127,0.860215,0.139785,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,0


# Feature Selection

## L1 Feature Selection

In [10]:
#Attempting Lasso(L1) feature selection
# shares data is not needed for classification
X = df.drop(labels=['popularity'], axis = 1, inplace=False)
y = df['popularity']
X= pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 666)



# Standardize the train and test sample
scaler = StandardScaler()
X_std = scaler.fit_transform(X_train)

# Perform GridSearchCV to tune best-fit LR model
param = {'C': [10**-2,10**-1,10**0,10**1,10**2]}

lr_model = LogisticRegression(penalty='l1', solver='liblinear')
lr_model.fit(X_train, y_train)


print("Redundant Feature Count: ", sum(lr_model.coef_[0]==0))
print("Redundant Feature Names: ", list(pd.Series(X_train.columns)[list(lr_model.coef_[0]==0)]))


Redundant Feature Count:  4
Redundant Feature Names:  [' global_rate_positive_words', ' global_rate_negative_words', ' avg_negative_polarity', ' max_negative_polarity']


In [11]:
df.drop(labels=[' global_rate_negative_words', ' avg_positive_polarity', ' global_sentiment_polarity', ' avg_negative_polarity', ' max_negative_polarity'], axis = 1, inplace=True)

## PCA Feature Reduction

## Fisher Score Feature Selection

In [12]:
# score = fisher_score.fisher_score(X_std, y_train)

# feat_importance = pd.Series(scores, X_std.colums[:])
# feat_importance.plot(kind='barh', color = 'teal')
# plt.show()
# print(score)

# Model Training

In [13]:
X = df.drop(labels=['popularity'], axis = 1, inplace=False)
y = df['popularity']
X= pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 666)

## Support Vector Machine

In [14]:
# svm = SVC(gamma='auto')
# svm.fit(X_train, y_train)

## Logistic Regression

In [15]:
# logistic_regression = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=6000)
# logistic_regression.fit(X_train, y_train)


## Random Forest

In [16]:
rf_classfication = RandomForestClassifier(n_estimators=1000, n_jobs=-1, max_depth=50,random_state=0)
rf_classfication.fit(X_train, y_train)

RandomForestClassifier(max_depth=50, n_estimators=1000, n_jobs=-1,
                       random_state=0)

# Evaluation

In [17]:
#Evaluateing Logistic Regression
# y_pred = logistic_regression.predict(X_test)
# print("F1 Score: ",f1_score(y_test, y_pred, average='macro'))

In [18]:
#Evaluating RF
y_pred = rf_classfication.predict(X_test)
print("F1 Score: ",f1_score(y_test, y_pred, average='macro'))

F1 Score:  0.3890139052124322


In [19]:
#Evaluating SVM
# y_pred = svm.predict(X_test)
# print("F1 Score: ",f1_score(y_test, y_pred, average='weighted'))

# Conclusions and Findings